# Solution for Puzzle x

## Imports

In [80]:
import pandas as pd
import numpy as np
import pulp
import os
from typing import List, Union
from tqdm import tqdm

## Input Data

In [84]:
def read_and_parse_irregular_data(file_path: str) -> pd.DataFrame:
    """
    Reads an irregularly formatted text file line by line, parses the data 
    blocks using string splitting, and returns a consolidated Pandas DataFrame.

    Args:
        file_path: The path to the input file (e.g., 'input/test.csv').

    Returns:
        A pandas DataFrame containing the parsed data.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"Error: File not found at '{file_path}'. Please check the path.")
        
    raw_lines = []
    try:
        # Read the entire file content into a list of strings
        with open(file_path, 'r', encoding='utf-8') as file:
            # Strip whitespace and filter out empty lines
            raw_lines = [line.strip() for line in file if line.strip()]
    except Exception as e:
        raise RuntimeError(f"An error occurred while reading the file: {e}")

    parsed_data = []

    # Parse each line string
    for line in raw_lines:

        # Split the line by spaces to separate the data blocks
        parts = line.split()
        
        # Check if the line contains enough data blocks
        if len(parts) < 3:
            print(f"Skipping line due to insufficient data: {line}")
            continue

        light_diagram = parts[0]
        joltage_requirements = parts[-1]
        buttons = parts[1:-1]
        
        # Prepare the data dictionary for the row
        row_dict = {'light_diagram': light_diagram, 'joltage_requirements': joltage_requirements, 'buttons': buttons}
        
        parsed_data.append(row_dict)

    # 3. Create the final DataFrame from the list of dictionaries
    df = pd.DataFrame(parsed_data)
    
    return df

file_to_process = 'input/day_10.csv'
input = read_and_parse_irregular_data(file_to_process)

print("\n--- Resulting DataFrame ---")
print(input.head(5))


--- Resulting DataFrame ---
  light_diagram    joltage_requirements  \
0      [#...##]  {132,30,23,13,121,115}   
1        [##.#]            {2,18,18,20}   
2      [.##...]     {38,30,29,22,13,25}   
3      [#..###]      {22,51,19,63,22,1}   
4      [#.#.##]     {13,43,33,29,40,41}   

                                             buttons  
0         [(0,1,3,4,5), (0,4,5), (1,2,3,4), (0,1,2)]  
1                                [(0,3), (1), (2,3)]  
2    [(0,1,5), (0,2,3,5), (0,3,4), (1,2), (0,1,2,4)]  
3  [(0,5), (1,3), (1,3,4), (0,1,2,3), (3), (0,1,3...  
4  [(2,3,4,5), (0,2,4,5), (0,1,4), (1,5), (1,2,3,...  


## Part One

In [57]:
def apply_button(status, button):
    """Applies button on all lamps in status."""
    button = button[1:-1] #strip ()
    lamps = button.split(",")
    for lamp in lamps:
        status[int(lamp)] *= -1
    return status

def bfs(curr_status, goal_status, buttons):
    """Performs BFS where buttons are vertices and lamp status lists the nodes."""
    seen = set()
    stack = [(curr_status, 0)]
    while stack:
        curr, curr_count = stack.pop(0)
        if curr == goal_status:
            return curr_count
        for button in buttons:
            new_status = apply_button(curr.copy(), button)
            if tuple(new_status) not in seen:
                seen.add(tuple(new_status))
                stack.append((new_status, curr_count+1))
    return -1

number_buttons_pressed = []
for idx, row in tqdm(input.iterrows(), total=len(input)):
    status_lights = [-1 for _ in range(len(row["light_diagram"])-2)]
    status_lights_goal = [-1 if status == "." else 1 for status in row["light_diagram"][1:-1]]
    buttons = row["buttons"]
    number_buttons_pressed_row = bfs(status_lights, status_lights_goal, buttons)
    number_buttons_pressed.append(number_buttons_pressed_row)
print(f"The fewest button presses required are {sum(number_buttons_pressed)}")
        

100%|██████████| 187/187 [00:00<00:00, 1498.50it/s]

The fewest button presses required are 491


## Part Two

In [83]:
# Too slow because the state space explodes

def apply_button(status, button):
    """Applies button on all lamps in status."""
    button = button[1:-1] #strip ()
    levers = button.split(",")
    for lever in levers:
        status[int(lever)] += 1
    return status

def bfs(curr_status, goal_status, buttons):
    """Performs BFS where buttons are vertices and joltage levels the nodes."""
    stack = [(curr_status, 0)]
    seen = set()
    while stack:
        curr, curr_count = stack.pop(0)
        if curr == goal_status:
            return curr_count
        for button in buttons:
            new_status = apply_button(curr.copy(), button)
            levels_valid = True
            # check if at least least one level is already too high
            for i in range(len(new_status)):
                if new_status[i] > goal_status[i]:
                    levels_valid = False
                    break
            if levels_valid and tuple(new_status) not in seen:
                seen.add(tuple(new_status))
                stack.append((new_status, curr_count+1))
    return -1

number_buttons_pressed = []
for idx, row in tqdm(input.iterrows(), total=len(input)):
    joltage_levels_goal = [int(level) for level in row["joltage_requirements"][1:-1].split(",")]
    joltage_levels = [0 for _ in range(len(joltage_levels_goal))]
    buttons = row["buttons"]
    number_buttons_pressed_row = bfs(joltage_levels, joltage_levels_goal, buttons)
    number_buttons_pressed.append(number_buttons_pressed_row)
print(f"The fewest button presses required are {sum(number_buttons_pressed)}")

100%|██████████| 3/3 [00:00<00:00, 56.14it/s]

The fewest button presses required are 33


In [86]:
def parse_goal_vector(goal_str: str) -> List[int]:
    """Parses the goal string '{3,5,4,7}' into a list of integers [3, 5, 4, 7]."""
    return [
        int(level.strip()) 
        for level in goal_str.strip('{}').split(',')
    ]

def solve_machine(goal_g_str: str, buttons_def: List[str]) -> Union[int, float]:
    """
    Solves the problem as a pure Integer Linear Programming (ILP) problem using PuLP.
    """
    goal_g = parse_goal_vector(goal_g_str)
    N = len(goal_g)       # Number of counters
    M = len(buttons_def)  # Number of buttons

    prob = pulp.LpProblem("Minimize Button Presses", pulp.LpMinimize)
    
    # x_i
    press_vars = [
        pulp.LpVariable(f"x_{i}", lowBound=0, cat='Integer')
        for i in range(M)
    ]
    prob += pulp.lpSum(press_vars), "Total Presses"

    A = np.zeros((N, M), dtype=np.int8)
    for col_index, button_str in enumerate(buttons_def):
        indices_str = button_str.strip('()')
        if not indices_str: continue 
        counter_indices = [int(i.strip()) for i in indices_str.split(',')]
        for row_index in counter_indices:
            if 0 <= row_index < N:
                A[row_index, col_index] = 1

    # A * x = G
    for row_index in range(N):
        equation_terms = [
            A[row_index, col_index] * press_vars[col_index] 
            for col_index in range(M)
        ]
        prob += pulp.lpSum(equation_terms) == goal_g[row_index], f"Joltage_Eq_{row_index}"

    prob.solve()

    if prob.status == pulp.LpStatusOptimal:
        return int(pulp.value(prob.objective))
    return np.inf

number_buttons_pressed = []

for _, row in input.iterrows():
    min_presses = solve_machine(
        goal_g_str=row["joltage_requirements"],
        buttons_def=row["buttons"]
    )
    number_buttons_pressed.append(min_presses)

valid_presses = [p for p in number_buttons_pressed if p != np.inf]

print(f"The fewest button presses required are {sum(valid_presses)}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/floriandreyer/Library/Mobile Documents/com~apple~CloudDocs/Programmierprojekte/AdventOfCode_2025/.venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/kj/q_05bvdx3s9crpx2lctmxkdr0000gn/T/a084e38800a34b8193d67a519cc6eee8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/kj/q_05bvdx3s9crpx2lctmxkdr0000gn/T/a084e38800a34b8193d67a519cc6eee8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 39 RHS
At line 46 BOUNDS
At line 51 ENDATA
Problem MODEL has 6 rows, 4 columns and 15 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 138 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objectiv